In [2]:
!pip install icechunk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 69.7 MB/s eta 0:00:00ta 0:00:01


In [16]:
import xarray as xr
import icechunk

icechunk_store = icechunk.s3_storage(
    bucket="cadcache", 
    prefix="wrf/ucla/cesm2/ssp370/1hr/t2/d03",
    from_env=True, # Get AWS credentials automatically from your local env  
    region='us-east-2', # Needs to match the region of the bucket
)

# Open the Icechunk repository for reading
# IMPORTANT: You need to provide credentials for the virtual chunks' actual location
# The virtual references point to s3://cadcat/, which is a public bucket
# Note: We need to match the exact prefix path that the virtual chunks use
credentials = icechunk.containers_credentials({
    "s3://cadcat/wrf/ucla/cesm2/ssp370/1hr/t2/d03/": icechunk.s3_credentials(anonymous=True)
})
read_repo = icechunk.Repository.open(icechunk_store, authorize_virtual_chunk_access=credentials)
read_session = read_repo.readonly_session("main")

# Open as xarray dataset using zarr
ds = xr.open_zarr(read_session.store, consolidated=False)
ds

<xarray.Dataset> Size: 360GB
Dimensions:            (y: 492, x: 243, time: 753360)
Coordinates:
    Lambert_Conformal  int32 4B ...
    lakemask           (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
    lat                (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
  * y                  (y) float64 4kB 4.549e+05 4.579e+05 ... 1.928e+06
    lon                (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
  * x                  (x) float64 2kB -4.335e+06 -4.332e+06 ... -3.609e+06
    landmask           (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
  * time               (time) datetime64[ns] 6MB 2014-09-01 ... 2100-08-31T23...
Data variables:
    t2                 (time, y, x) float32 360GB dask.array<chunksize=(14541, 69, 33), meta=np.ndarray>
Attributes: (12/94)
    AERCU_FCT:                       1.0
    AERCU_OPT:                       0
    AUTO_LEVELS_OPT:                 2
    BL_PBL_PHYSICS:                  1
    BOTTOM-TOP_GRID_DIMENSION:       40
    BOTTOM-TOP_PATCH_END_STAG:       40
    ...                              ...
    frequency:                       1hr
    institution:                     UCLA Center for Climate Science
    nominal_resolution:              3 km
    source:                          https://dept.atmos.ucla.edu/alexhall/dow...
    source_id:                       cesm2
    variant_label:                   r11i1p1f1

In [17]:
%%time
test_mean = ds.mean().compute()

CPU times: user 19min 44s, sys: 15min 29s, total: 35min 14s
Wall time: 11min 7s


In [18]:
test_mean

<xarray.Dataset> Size: 8B
Dimensions:            ()
Coordinates:
    Lambert_Conformal  int32 4B 1
Data variables:
    t2                 float32 4B 289.1
Attributes: (12/94)
    AERCU_FCT:                       1.0
    AERCU_OPT:                       0
    AUTO_LEVELS_OPT:                 2
    BL_PBL_PHYSICS:                  1
    BOTTOM-TOP_GRID_DIMENSION:       40
    BOTTOM-TOP_PATCH_END_STAG:       40
    ...                              ...
    frequency:                       1hr
    institution:                     UCLA Center for Climate Science
    nominal_resolution:              3 km
    source:                          https://dept.atmos.ucla.edu/alexhall/dow...
    source_id:                       cesm2
    variant_label:                   r11i1p1f1

In [2]:
import climakitae as ck
cd = ck.ClimateData()

climakit_dataset = (
    cd 
    .catalog("cadcat")
    .activity_id("WRF")
    .institution_id("UCLA")
    .source_id("CESM2")
    .experiment_id("ssp370")
    .variable("t2")
    .grid_label("d03")
    .table_id("1hr")
    .processes({
        "filter_unadjusted_models": "no",
        "concat": "sim"
    })
    .get()
)
climakit_dataset = climakit_dataset.isel(sim=0)
climakit_dataset

2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Initializing ClimateData interface
2025-11-14 18:44:08 - climakitae.new_core.dataset_factory - INFO - DatasetFactory initialized with 2 validators and 7 processors
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - ClimateData initialization successful
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - ✅ Ready to query!
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Catalog set to: cadcat
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Activity ID set to: WRF
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Institution ID set to: UCLA
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Source ID set to: CESM2
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Experiment ID(s) set to: ['ssp370']
2025-11-14 18:44:08 - climakitae.new_core.user_interface - INFO - Variable set to: t2
2025-11-14 18:44:08 - climakita

<xarray.Dataset> Size: 360GB
Dimensions:            (time: 753360, y: 492, x: 243)
Coordinates:
    sim                <U31 124B 'wrf_ucla_cesm2_ssp370_r11i1p1f1'
    Lambert_Conformal  int32 4B ...
    lakemask           (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
    landmask           (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
    lat                (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
    lon                (y, x) float32 478kB dask.array<chunksize=(69, 33), meta=np.ndarray>
  * time               (time) datetime64[ns] 6MB 2014-09-01 ... 2100-08-31T23...
  * x                  (x) float64 2kB -4.335e+06 -4.332e+06 ... -3.609e+06
  * y                  (y) float64 4kB 4.549e+05 4.579e+05 ... 1.928e+06
Data variables:
    t2                 (time, y, x) float32 360GB dask.array<chunksize=(14541, 69, 33), meta=np.ndarray>
Attributes: (12/111)
    AERCU_FCT:                        1.0
    AERCU_OPT:                        0
    AUTO_LEVELS_OPT:                  2
    BL_PBL_PHYSICS:                   1
    BOTTOM-TOP_GRID_DIMENSION:        40
    BOTTOM-TOP_PATCH_END_STAG:        40
    ...                               ...
    intake_esm_attrs:_data_format_:   zarr
    intake_esm_dataset_key:           WRF.UCLA.CESM2.ssp370.1hr.d03
    resolution:                       3 km
    filter_unadjusted_models:         no
    concat:                           Process 'concat' applied to the data. M...
    update_attributes:                Process 'update_attributes' applied to ...

In [ ]:
%%time
ck_test_mean = climakit_dataset.mean().compute()

In [ ]:
ck_test_mean

In [1]:

time: 297840y: 492x: 243

SyntaxError: invalid decimal literal (<unknown>, line 1)

In [ ]:

y: 492x: 243time: 753360